In [ ]:
import cv2
import imutils
from imutils import face_utils
import dlib
from scipy.spatial import distance
from pygame import mixer
import threading

def play_alert_sound():
    mixer.init()
    mixer.music.load('mixkit-residential-burglar-alert-1656.wav')
    mixer.music.play()

def eye_aspect_ratio(eye):
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

def detect_yawn(mouth):
    mouth_width = distance.euclidean(mouth[0], mouth[6])
    lip_distance = distance.euclidean(mouth[2], mouth[10])
    aspect_ratio = lip_distance / mouth_width
    return aspect_ratio

thresh_eye = 0.25
thresh_aspect_ratio = 0.5
frame_check = 20
flag_eye = 0
flag_yawn = 0
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS['left_eye']
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS['right_eye']
(mStart, mEnd) = face_utils.FACIAL_LANDMARKS_IDXS['mouth']

detect = dlib.get_frontal_face_detector()
predict = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    subjects = detect(gray, 0)

    for subject in subjects:
        shape = predict(gray, subject)
        shape = face_utils.shape_to_np(shape)
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        mouth = shape[mStart:mEnd]
        leftEar = eye_aspect_ratio(leftEye)
        rightEar = eye_aspect_ratio(rightEye)
        ear = (leftEar + rightEar) / 2.0

        leftEyeHull = cv2.convexHull(leftEye.astype(int))
        rightEyeHull = cv2.convexHull(rightEye.astype(int))
        mouthHull = cv2.convexHull(mouth)

        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [mouthHull], -1, (0, 255, 0), 1)

        if ear < thresh_eye:
            flag_eye += 1
            if flag_eye >= frame_check:
                cv2.putText(frame, 'Alert: Drowsiness Detected', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7,
                            (0, 0, 255), 2)
                cv2.putText(frame, 'Alert: Drowsiness Detected', (10, 325), cv2.FONT_HERSHEY_SIMPLEX, 0.7,
                            (0, 0, 255), 2)
                threading.Thread(target=play_alert_sound).start()
        else:
            flag_eye = 0

        aspect_ratio = detect_yawn(mouth)
        if aspect_ratio > thresh_aspect_ratio:
            flag_yawn += 1
            if flag_yawn >= frame_check:
                cv2.putText(frame, 'Alert: Yawning Detected', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7,
                            (0, 0, 255), 2)
                cv2.putText(frame, 'Alert: Yawning Detected', (10, 355), cv2.FONT_HERSHEY_SIMPLEX, 0.7,
                            (0, 0, 255), 2)
                threading.Thread(target=play_alert_sound).start()
        else:
            flag_yawn = 0

    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()